# Artificial and Computational Intelligence (ACI): Assignment 1

#### Assignment Contributor



This assignment is submitted by: **Group 306**.

The team includes:

| S.no.          | Name           | BITS ID        | Contribution   |
|----------------|----------------|----------------|----------------|
| 1              | Vidushi Bhatia | 2024AC05012    | 100%           |

<br>

<hr>

<br>

#### Assigned Problem Statement

Given the below maze configuration, the task of the robot is to navigate in the maze and find
the optimal path to reach the finish position. It can move to the north, south, west and east
direction. While navigating through the environment it has obstacles like walls. For each
transition, a path cost of +3 is added in search. Assume that the robot’s vision sensors are
sensitive to the exposure to the sunlight and whenever it tries to move towards the east direction
resulting in incurring an additional penalty of +5 cost. Use Manhattan distance as a heuristic
wherever necessary.

<img src = "maze-problem.png" width = 50%>

<br>

a. Explain the PEAS (Performance measure, Environment, Actuator, Sensor.) for
your agent. (2 Marks)

b. Implement A* algorithm using python. Use the below combination and
interpret your observations (6 Marks)

`f(n) = g(n) + w.h(n)`

Scenario 1
- Heuristic: Manhattan Distance
- Heuristic Weight (w): 1.0
- Observation Focus: Optimal path guaranteed, slower due to full exploration.

Scenario 2
- Heuristic: Manhattan Distance
- Heuristic Weight (w): 2.5
- Observation Focus: Faster search with fewer nodes expanded, but path may be suboptimal

Carefully read the question and submit your individual response using this
form: (5 Marks)

<br>

<hr>

<br>

## Problem Solution

#### 1.	Define the environment in the following block

<u>**PEAS Description**</u>

**P - Performance Metrics**
- Total path cost - Use the most efficient (optimal) path according to the defined costs. (minimize)
- Number of collisions with walls. (minimize)
- Number of steps to reach the finish point. (minimize)
- Number of steps taken in the east direction - towards the sun. (minimize)
- Time taken to find the finish position. (minimize)

**E - Environment**
- A maze with gird layout.
- Clearly defined start and goal positions.
- Fixed walls that act as obstacles and block movement.
- Sunlight exposure on the east.
- Static, observable layout.
- No dynamic obstacles, deterministic.


**A - Actuators**
- Ability to traverse on open paths - robot legs/wheels.
- Ability to change direction / steering ability. (4 directions)
- Wall-avoidance mechanism (halt on blocked direction).

**S - Sensors**
- Sensors to detect walls (Proximity/vision).
- Sensors to dectect sunlight (thermal/UV).
- Position awareness within the grid (GPS/odometry).

<br>
<hr>
<br>

<u>**Given Problem Formulation**</u>

**Initial State**
The agent starts at the given Start position.

**Actions**
- move: north, south, west and east direction
- can't move through obstacles like walls

**State Space**
 State Representation -- A state is represented as the current cell coordinates: (x, y).

**Transition Model**
Given a state (x, y) and an action, the agent transitions to a new state (x', y') if the move is legal.
	•	Each state represents the robot’s current position (row, column) in the grid.
    •	The robot can be in any free cell that is not blocked by a wall within the maze boundaries.

**Goal Test**
Find the optimal path to reach the finish position from the given initial state with minimum path cost and minimum manhnttan distance heuristic.
The goal test checks if the current state equals the goal coordinates.


**Path Cost**
- for each transition, +3
- for each move towards east, additional +5

<br>
<hr>
<br>



#### 2.	Util Functions for Initial State, Path Cost Computation and Successors

In [1]:
import time
import heapq
import sys
import tracemalloc

In [2]:
def is_valid(pos):
        return 0 <= pos[0] < rows and 0 <= pos[1] < cols and pos in maze

def set_initial_state(rows, cols, maze, start_input, goal_input):    
    if not is_valid(start_input):
        raise ValueError(f"Invalid start position: {start_input}")
    if not is_valid(goal_input):
        raise ValueError(f"Invalid goal position: {goal_input}")
    if start_input == goal_input:
        raise ValueError("Start and goal cannot be the same.")

    return {
        'start': start_input,
        'goal': goal_input,
        'current': start_input
    }

In [3]:
def manhattan(current, goal):
    return abs(current[0] - goal[0]) + abs(current[1] - goal[1])


In [4]:
def get_successors(cell, adj_map, goal):
    x, y = cell
    directions = {
        0: (-1, 0),  # North
        1: (1, 0),   # South
        2: (0, 1),   # East (sunlight penalty)
        3: (0, -1)   # West
    }

    successors = []
    for i in range(4):
        if adj_map[cell][i]:  # direction is open
            dx, dy = directions[i]
            neighbor = (x + dx, y + dy)
            if neighbor in adj_map:
                move_cost = 3 + (5 if i == 2 else 0)  # East penalty
                successors.append((neighbor, move_cost))
    return successors

In [5]:
def is_goal(state, goal):
    return state == goal

#### 3.	A* Algorithm
`f(n) = g(n) + w.h(n)`

In [6]:
def a_star_search(adj_map, start, goal, w=1.0):
    open_list = []
    heapq.heappush(open_list, (0, 0, start))  # (f, g, state)
    came_from = {}
    cost_so_far = {start: 0}
    nodes_expanded = 0

    while open_list:
        print(f"\n--- Step {nodes_expanded + 1} ---")
        print("Open List:")
        for f, g, node in open_list:
            print(f"  State: {node}, g(n): {g}, h(n): {manhattan(node, goal)}, f(n): {f:.1f}")

        _, g, current = heapq.heappop(open_list)
        nodes_expanded += 1
        print(f"\nExpanding: {current}")

        if is_goal(current, goal):
            path = reconstruct_path(came_from, current)
            return {
                'path': path,
                'cost': cost_so_far[current],
                'nodes_expanded': nodes_expanded
            }

        for neighbor, move_cost in get_successors(current, adj_map, goal):
            new_cost = cost_so_far[current] + move_cost
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                h = manhattan(neighbor, goal)
                f = new_cost + w * h
                heapq.heappush(open_list, (f, new_cost, neighbor))
                came_from[neighbor] = current
                print(f"  → Considering: {neighbor}, g(n): {new_cost}, h(n): {h}, f(n): {f:.1f}")

    return None

In [7]:
def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    return path[::-1]

#### 4. Define Initial State with Dynamic Input

In [8]:
## MAZE INPUT

# (cell): [N, S, E, W] 

# (r, c): [N,S,E,W]	(2,3): [1,0,1,0]
# 0 means blocked

maze = {
    (0, 0): [0, 1, 1, 0],
    (0, 1): [0, 1, 1, 1],
    (0, 2): [0, 1, 0, 1],
    (0, 3): [0, 1, 1, 0],
    (0, 4): [0, 1, 0, 1],
    (0, 5): [0, 1, 1, 0],
    (0, 6): [0, 1, 0, 1],

    (1, 0): [1, 1, 0, 0],
    (1, 1): [1, 1, 0, 0],
    (1, 2): [1, 1, 0, 0],
    (1, 3): [1, 1, 0, 0],
    (1, 4): [1, 1, 0, 0],
    (1, 5): [1, 1, 0, 0],
    (1, 6): [1, 1, 0, 0],

    (2, 0): [1, 0, 0, 0],
    (2, 1): [1, 1, 0, 0],
    (2, 2): [1, 1, 0, 0],
    (2, 3): [1, 1, 0, 0],
    (2, 4): [1, 1, 0, 0],
    (2, 5): [1, 1, 0, 0],
    (2, 6): [1, 1, 0, 0],

    (3, 0): [0, 1, 1, 1], # START
    (3, 1): [1, 0, 0, 1],
    (3, 2): [1, 1, 0, 0],
    (3, 3): [1, 1, 0, 0],
    (3, 4): [1, 1, 0, 0],
    (3, 5): [1, 1, 0, 0],
    (3, 6): [1, 1, 1, 0], # FINISH

    (4, 0): [1, 1, 0, 0],
    (4, 1): [0, 0, 1, 0],
    (4, 2): [1, 0, 1, 1],
    (4, 3): [1, 0, 0, 1],
    (4, 4): [1, 0, 1, 0],
    (4, 5): [1, 0, 0, 1],
    (4, 6): [1, 1, 0, 0],

    (5, 0): [1, 0, 1, 0],
    (5, 1): [0, 0, 1, 1],
    (5, 2): [0, 0, 1, 1],
    (5, 3): [0, 0, 1, 1],
    (5, 4): [0, 0, 1, 1],
    (5, 5): [0, 0, 1, 1],
    (5, 6): [1, 0, 0, 1]
}

In [9]:
rows = 5
cols = 6
start = (0, 0)
goal = (4, 5)
initial_state = set_initial_state(rows, cols, maze, start, goal)

#### 5. Run algorithm with Dynamic Input

Scenario 1
- Heuristic: Manhattan Distance
- Heuristic Weight (w): 1.0
- Observation Focus: Optimal path guaranteed, slower due to full exploration.

In [10]:
tracemalloc.start()
start_time = time.time()
result_1 = a_star_search(
    adj_map=maze,
    start=initial_state['start'],
    goal=initial_state['goal'],
    w=1.0  # Or w=2.5 for faster (but suboptimal) path
)
end_time = time.time()
execution_time_1 = end_time - start_time
current_1, peak_1 = tracemalloc.get_traced_memory()
tracemalloc.stop()


--- Step 1 ---
Open List:
  State: (0, 0), g(n): 0, h(n): 9, f(n): 0.0

Expanding: (0, 0)
  → Considering: (1, 0), g(n): 3, h(n): 8, f(n): 11.0
  → Considering: (0, 1), g(n): 8, h(n): 8, f(n): 16.0

--- Step 2 ---
Open List:
  State: (1, 0), g(n): 3, h(n): 8, f(n): 11.0
  State: (0, 1), g(n): 8, h(n): 8, f(n): 16.0

Expanding: (1, 0)
  → Considering: (2, 0), g(n): 6, h(n): 7, f(n): 13.0

--- Step 3 ---
Open List:
  State: (2, 0), g(n): 6, h(n): 7, f(n): 13.0
  State: (0, 1), g(n): 8, h(n): 8, f(n): 16.0

Expanding: (2, 0)

--- Step 4 ---
Open List:
  State: (0, 1), g(n): 8, h(n): 8, f(n): 16.0

Expanding: (0, 1)
  → Considering: (1, 1), g(n): 11, h(n): 7, f(n): 18.0
  → Considering: (0, 2), g(n): 16, h(n): 7, f(n): 23.0

--- Step 5 ---
Open List:
  State: (1, 1), g(n): 11, h(n): 7, f(n): 18.0
  State: (0, 2), g(n): 16, h(n): 7, f(n): 23.0

Expanding: (1, 1)
  → Considering: (2, 1), g(n): 14, h(n): 6, f(n): 20.0

--- Step 6 ---
Open List:
  State: (2, 1), g(n): 14, h(n): 6, f(n): 20.0


In [11]:
if result_1:
    print("Final Solution:")
    print("→ Path:")
    for step in result_1['path']:
        print(f"   {step}")
    print(f"→ Total Cost: {result_1['cost']}")
    print(f"→ Total Steps: {len(result_1['path']) - 1}")
    print(f"→ Nodes Expanded: {result_1['nodes_expanded']}")
else:
    print("No path found.")

Final Solution:
→ Path:
   (0, 0)
   (0, 1)
   (0, 2)
   (1, 2)
   (2, 2)
   (3, 2)
   (4, 2)
   (4, 3)
   (3, 3)
   (2, 3)
   (1, 3)
   (0, 3)
   (0, 4)
   (1, 4)
   (2, 4)
   (3, 4)
   (4, 4)
   (4, 5)
→ Total Cost: 76
→ Total Steps: 17
→ Nodes Expanded: 33



Scenario 2
- Heuristic: Manhattan Distance
- Heuristic Weight (w): 2.5
- Observation Focus: Faster search with fewer nodes expanded, but path may be suboptimal



In [12]:
tracemalloc.start()
start_time = time.time()
result_2 = a_star_search(
    adj_map=maze,
    start=initial_state['start'],
    goal=initial_state['goal'],
    w=2.5
)
end_time = time.time()
execution_time_2 = end_time - start_time
current_2, peak_2 = tracemalloc.get_traced_memory()
tracemalloc.stop()


--- Step 1 ---
Open List:
  State: (0, 0), g(n): 0, h(n): 9, f(n): 0.0

Expanding: (0, 0)
  → Considering: (1, 0), g(n): 3, h(n): 8, f(n): 23.0
  → Considering: (0, 1), g(n): 8, h(n): 8, f(n): 28.0

--- Step 2 ---
Open List:
  State: (1, 0), g(n): 3, h(n): 8, f(n): 23.0
  State: (0, 1), g(n): 8, h(n): 8, f(n): 28.0

Expanding: (1, 0)
  → Considering: (2, 0), g(n): 6, h(n): 7, f(n): 23.5

--- Step 3 ---
Open List:
  State: (2, 0), g(n): 6, h(n): 7, f(n): 23.5
  State: (0, 1), g(n): 8, h(n): 8, f(n): 28.0

Expanding: (2, 0)

--- Step 4 ---
Open List:
  State: (0, 1), g(n): 8, h(n): 8, f(n): 28.0

Expanding: (0, 1)
  → Considering: (1, 1), g(n): 11, h(n): 7, f(n): 28.5
  → Considering: (0, 2), g(n): 16, h(n): 7, f(n): 33.5

--- Step 5 ---
Open List:
  State: (1, 1), g(n): 11, h(n): 7, f(n): 28.5
  State: (0, 2), g(n): 16, h(n): 7, f(n): 33.5

Expanding: (1, 1)
  → Considering: (2, 1), g(n): 14, h(n): 6, f(n): 29.0

--- Step 6 ---
Open List:
  State: (2, 1), g(n): 14, h(n): 6, f(n): 29.0


In [13]:
if result_2:
    print("Final Solution:")
    print("→ Path:")
    for step in result_2['path']:
        print(f"   {step}")
    print(f"→ Total Cost: {result_2['cost']}")
    print(f"→ Total Steps: {len(result_2['path']) - 1}")
    print(f"→ Nodes Expanded: {result_2['nodes_expanded']}")
else:
    print("No path found.")

Final Solution:
→ Path:
   (0, 0)
   (0, 1)
   (0, 2)
   (1, 2)
   (2, 2)
   (3, 2)
   (4, 2)
   (4, 3)
   (3, 3)
   (2, 3)
   (1, 3)
   (0, 3)
   (0, 4)
   (1, 4)
   (2, 4)
   (3, 4)
   (4, 4)
   (4, 5)
→ Total Cost: 76
→ Total Steps: 17
→ Nodes Expanded: 32


#### 5.	Compute Time and Space Consumed

In [14]:
## Time Taken
print(f"Time Taken for first run: {execution_time_1:.6f} seconds")
print(f"Time Taken for first run: {execution_time_2:.6f} seconds")


Time Taken for first run: 0.013045 seconds
Time Taken for first run: 0.014627 seconds


In [16]:
## Peak memory consumed
print(f"Peak Memory Usage for first run: {peak_1 / 1024:.2f} KB")
print(f"Peak Memory Usage for second run: {peak_2 / 1024:.2f} KB")


Peak Memory Usage for first run: 50.72 KB
Peak Memory Usage for second run: 50.04 KB
